In [1]:
import jax
from typing import Any, Callable, Sequence
from jax import random, numpy as jnp
import flax
from flax import linen as nn
import jax.numpy as jnp
import tensorflow as tf
import tensorflow_datasets as tfds

from clu import metrics
from flax.training import train_state  # Useful dataclass to keep train state
from flax import struct                # Flax dataclasses
import optax                           # Common loss functions and optimizers


2024-03-03 16:15:45.752229: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-03 16:15:45.752255: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-03 16:15:45.753219: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-03 16:15:46.303504: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import tensorflow_datasets as tfds


In [7]:
ds = tfds.load('wikipedia')

In [8]:
ds['train']

<_PrefetchDataset element_spec={'text': TensorSpec(shape=(), dtype=tf.string, name=None), 'title': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [10]:

@tf.py_function(Tout=tf.int32)
def tokenize_py_func(text):
    # Convert tensor to numpy array
    text = text.numpy().decode()
    # Tokenize using Hugging Face tokenizer
    return tokenizer.encode(text, return_tensors="tf")

tokenizer = AutoTokenizer.from_pretrained("openai-gpt")

BATCH_SIZE = 32
VOCAB_SIZE = tokenizer.vocab_size

d = tfds.load('wikipedia')['train']
d = (d
     .map(lambda x: x['text'], num_parallel_calls=tf.data.AUTOTUNE)
     .map(tokenize_py_func, num_parallel_calls=tf.data.AUTOTUNE)
     .map(lambda x: {'x': x[0][:-1], 'y': tf.roll(x[0], -1, 0)[:-1]}, num_parallel_calls=tf.data.AUTOTUNE)
     .unbatch()
     .shuffle(1024)
     .batch(BATCH_SIZE)
     .prefetch(1)
    )

NameError: name 'AutoTokenizer' is not defined

In [551]:
for e in d.take(5):
    print(e)

Token indices sequence length is longer than the specified maximum sequence length for this model (2376 > 512). Running this sequence through the model will result in indexing errors


{'x': <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([ 2720, 14085,   481, 14085,  7649,   544,  8312,   244, 10945,
        5722,   485,   488,  4640,   488,  4240,   729,   240,   872,
         481,  7598,   488, 32965,   239,  2672, 20733,   260, 26836,
       36792,  3690,  7649, 15959, 22772], dtype=int32)>, 'y': <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([  260,  8312,  3779,   270, 13057,   604,   485,   240,   498,
        1780, 35351,   246, 16667,  1964,   617,  2216,   488,   498,
        2720, 19290,  7271,   275, 39475, 14319, 15776,  5699,     0,
         500,   872, 24476,  7271,   831], dtype=int32)>}
{'x': <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([  725, 33410,  4353,   725, 35351,   556,  6350,   928,  1026,
       15827,   500, 34575,   630,  2445, 33490,  1474,   562,   616,
         568,   522,  1147,   246,   481,   481,   765, 19155,  3706,
       15018, 35351,   500,   485, 15186], dtype=int32)>, 'y': <tf.Tensor: shape=(32,), dtype=int32, n

In [ ]:
import flax.linen as nn
import jax, jax.numpy as jnp


In [208]:
layer = nn.Embed(num_embeddings=VOCAB_SIZE, features=64)
params = layer.init(jax.random.key(0), jnp.ones((1, 5), dtype=int))
jax.tree_util.tree_map(lambda x: x.shape, params) 

{'params': {'embedding': (40478, 64)}}

In [248]:
class UnigramModel(nn.Module):
    vocab_size: int
    embedding_dim: int
        
    @nn.compact
    def __call__(self, x):
        x = nn.Embed(num_embeddings=self.vocab_size, features=self.embedding_dim, name="unigram")(x)
        x = nn.Dense(features=self.vocab_size)(x)
        return x
        

In [249]:
model = UnigramModel(vocab_size=VOCAB_SIZE, embedding_dim=64)
key1, key2 = random.split(random.key(0))


In [308]:
@struct.dataclass
class Metrics(metrics.Collection):
    accuracy: metrics.Accuracy
    loss: metrics.Average.from_output('loss')

class TrainState(train_state.TrainState):
    metrics: Metrics

def create_train_state(module, rng, learning_rate, momentum):
    """Creates an initial `TrainState`."""
    dummy = random.randint(key1, minval=0, maxval=10, shape=(10,))
    params = module.init(rng, dummy)['params'] # initialize parameters by passing a template image
    tx = optax.sgd(learning_rate, momentum)
    return TrainState.create(
      apply_fn=module.apply, params=params, tx=tx,
      metrics=Metrics.empty())


In [515]:
@jax.jit
def train_step(state, batch):
    """Train for a single step."""
    def loss_fn(params):
        logits = state.apply_fn({'params': params}, batch['x'])
        loss = optax.softmax_cross_entropy_with_integer_labels(
            logits=logits, labels=batch['y']).mean()
        return loss
    grad_fn = jax.grad(loss_fn)
    grads = grad_fn(state.params)
    state = state.apply_gradients(grads=grads)
    return state

@jax.jit
def compute_metrics(*, state, batch):
    logits = state.apply_fn({'params': state.params}, batch['x'])
    loss = optax.softmax_cross_entropy_with_integer_labels(
        logits=logits, labels=batch['y']).mean()
    metric_updates = state.metrics.single_from_model_output(
    logits=logits, labels=batch['y'], loss=loss)
    metrics = state.metrics.merge(metric_updates)
    state = state.replace(metrics=metrics)
    return state


@jax.jit
def pred_step(state, x, key, T=10):
    logits = state.apply_fn({'params': state.params}, x)
    return jax.random.categorical(key, logits/T, axis=1)

def generate(init_word, num_steps, key, T=10):
    w = init_word
    s = w
    for i in range(num_steps):
        key, _ = random.split(key)
        w = tokenizer.decode(pred_step(state, np.array(tokenizer.encode(w)), key, T))
        s += (' ' + w)
    return s

In [447]:
key = random.PRNGKey(0)



[1437271066   43563307]


In [552]:
learning_rate = 0.01
momentum = 0.9

init_rng = jax.random.key(0)
state = create_train_state(model, init_rng, learning_rate, momentum)
metrics_history = {'train_loss': [],
                   'train_accuracy': []}

num_steps = 100000
d_iter = d.as_numpy_iterator()
for step in range(num_steps):
    batch = d_iter.next()
    # Run optimization steps over training batches and compute batch metrics
    state = train_step(state, batch) # get updated train state (which contains the updated parameters)
    state = compute_metrics(state=state, batch=batch) # aggregate batch metrics

    if (step+1) % 1000 == 0: # one training epoch has passed
        for metric,value in state.metrics.compute().items(): # compute metrics
            metrics_history[f'train_{metric}'].append(value) # record metrics
        state = state.replace(metrics=state.metrics.empty()) # reset train_metrics for next training epoch

        print(f"step: {(step+1)}, "
          f"loss: {metrics_history['train_loss'][-1]}, "
          f"accuracy: {metrics_history['train_accuracy'][-1] * 100}")


KeyboardInterrupt: 

In [541]:
key = random.PRNGKey(1)
print(generate('hello', 20, key, T=2))


hello dir passionately deduce stank venice ceremonial integrity advertise donkey whichever lifting avi goo exchanges negro bean jonas mutton leaner iette


[3570]

In [290]:
jax.nnmodel.apply(params, x) * jax.nn.one_hot(y, VOCAB_SIZE)


Array([[-0.,  0.,  0., ...,  0., -0., -0.],
       [ 0.,  0.,  0., ...,  0., -0.,  0.],
       [ 0., -0.,  0., ..., -0., -0.,  0.],
       ...,
       [-0., -0.,  0., ...,  0.,  0.,  0.],
       [ 0., -0.,  0., ...,  0.,  0., -0.],
       [ 0., -0.,  0., ...,  0., -0., -0.]], dtype=float32)

Array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

(10, 40478)

Array([2, 1, 7, 7, 3, 5, 7, 8, 2, 2], dtype=int32)